In [1]:
import numpy as np
import pandas as pd
import os
import glob
import re
import textiling
import segmentation
import nltk
nltk.download('stopwords')
from sentence_transformers import SentenceTransformer

import konlpy
from konlpy.tag import Okt
import matplotlib.pylab as plt
%matplotlib inline


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/lfoyh6591/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
def load_data(name, version):
    path = name + "_" + version + "_data.txt"
    text = ""
    with open(path, "r") as f:
        lines = f.readlines()

    for line in lines:
        text += line

    return text

kakao_origin = load_data("kakao", "origin")
kakao_stem = load_data("kakao", "stem")
band_origin = load_data("band", "origin")
band_stem = load_data("band", "stem")

In [16]:
tt = textiling.TextTilingTokenizer()
kakao_cos_tt = tt.tokenize(kakao_origin, mode = "cos")
kakao_origin_tt = tt.tokenize(kakao_stem, mode = "origin")
band_cos_tt = tt.tokenize(band_origin, mode = "cos")
band_origin_tt = tt.tokenize(band_stem, mode = "origin")

TypeError: tokenize() got an unexpected keyword argument 'mod'

In [ ]:
def origin_pred(tt):
    origin_label = [0]
    for origin_t in tt:
        origin_label.append(origin_label[-1] + len(origin_t.strip().split("\n\n\t")))

    return origin_label[1:]

def cos_pred(tt):
    cos_label = [0]
    for cos_t in tt:
        cos_label.append(cos_label[-1] + len(cos_t.split("\n")))

    return cos_label[1:]

kakao_origin_pred = origin_pred(kakao_origin_tt)
kakao_cos_pred = cos_pred(kakao_cos_tt)
band_origin_pred = origin_pred(band_origin_tt)
band_cos_pred = cos_pred(band_cos_tt)

In [ ]:
def load_label(data_name):
    with open(data_name+"_label.txt") as f:
        label = [int(x) for x in f.readline().strip().split(" ")]
    return label

kakao_label = load_label("kakao")
band_label = load_label("band")

In [ ]:
def draw_graph(label, cos_pred, origin_pred):
    plt.figure(figsize=(20,2))
    for i in (label):
        plt.axvline(x = i, color="r", linewidth = 1)
    for i in (cos_pred):
        plt.axvline(x = i, color="b", linewidth = 1)
    for i in (origin_pred):
        plt.axvline(x = i, color="y", linewidth = 1)
    plt.legend()
    plt.show()

In [ ]:
draw_graph(kakao_label, kakao_cos_pred, kakao_origin_pred)

In [ ]:
draw_graph(band_label, band_cos_pred, band_origin_pred)

In [ ]:
def evaluate_windowdiff(label, pred, n):
    label_s = ""
    pred_s = ""
    for i in range(label[-1]+1):
        if i in label:
            label_s += "1"
        else:
            label_s += "0"

        if i in pred:
            pred_s += "1"
        else:
            pred_s += "0"

    return segmentation.windowdiff(label, pred, n)
